#1. Импортиртируем модули и загружаем данные

В НОУТБУКЕ ПРЕДСТАВЛЕНА ЛУЧШАЯ МОДЕЛЬ ИЗ РАССМОТРЕННЫХ С ЛУЧШИМИ ПАРАМЕТРАМИ. МЕНЯЛИСЬ КОЛ-ВО СЛОЕВ В СЕТИ, ДЛИНА СТАТЬИ, ДРОПАУТ. На первом этапе произведен импорт всех необходимых модулей, загруженны данный из гугл диска (ria_processed; предобученные вектора fasttext для русского языка). Данные приведены к формату data - кортеж, состоящий из двух списков. В первом - заголовки, во втором - статьи.

In [0]:
import ujson 
import re
import numpy as np
import pickle
import random, sys
from keras.preprocessing import sequence
from keras.utils import np_utils

Using TensorFlow backend.


In [0]:
#Данные скачиваются из гугл диска, поэтому нужно предоставить колабу доступ. 
#!pip install PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#id файлов с гугл диска. 
#Исходные файлы есть на гитхабе (https://github.com/RossiyaSegodnya/ria_news_dataset) и тут(https://fasttext.cc/docs/en/crawl-vectors.html)
download = drive.CreateFile({'id': '1SDJlygqwS_zj5EEeQU9LV4WCtwRvPivC'})
download.GetContentFile('ria.json.gz')
download = drive.CreateFile({'id': '1gvAL8sb8t5czyTPh84-pvBvq0Zake2_K'})
download.GetContentFile('cc.ru.300.vec.gz')

In [0]:
!gunzip 'cc.ru.300.vec.gz'
!gunzip 'ria.json.gz'

gzip: cc.ru.300.vec already exists; do you wish to overwrite (y or n)? n
	not overwritten
gzip: ria.json already exists; do you wish to overwrite (y or n)? n
	not overwritten


In [0]:
json_name = 'ria_1k.json'
json_full = 'ria.json'

In [0]:
data_json = []
with open(json_full, 'r', encoding = 'utf-8') as file:
            for line in file.readlines()[:50000]:
                data_json += [ujson.loads(line)]

In [0]:
type(data_json), len(data_json), data_json[0]

(list,
 50000,
 {'text': '<p><strong></strong></p>\n<p><strong>москва, 31 янв - риа новости.</strong> большая часть из 33 детей, которых граждане сша пытались вывезти из гаити в организованный в доминиканской республике приют, не являются сиротами, сообщает в воскресенье <a href="http://www.afp.com" target="_blank">агентство франс пресс</a> со ссылкой на заявление представителя международной организации "детские деревни sos" (sos children\'s village), оказывающей помощь детям, оставшимся без родителей</p>\n<p>как заявила агентству патрисия варгас (patricia vargas), курирующая программы "детских деревень sos" в центральной америке, мексике и на карибах, поговорив с детьми она выяснила, что родители многих из них живы. некоторые дети смогли назвать свои домашние адреса и номера телефонов, что дает возможность связаться с их родителями.</p>\n<p>в это воскресенье <a href="http://rian.ru/society/20100131/207037914.html" target="_blank">гаитянская полиция задержала десятерых граждан сша</a>,

In [0]:
#data_processed = []
heads = []
descs = []
keywords = []
pre_data = []
for val in data_json:
    title = val['title']
    title = re.sub(r'\<[^>]*\>', '', title)
    body = val['text']
    body = re.sub(r'\<[^>]*\>', '', body)
    body = re.sub(r'\n', ' ', body)
    body = re.sub(r'&[a-z]{0,7};', ' ', body)
    body = re.sub(r'\s{2,10}', ' ', body)
    title = re.sub(r'\s{2,10}', ' ', title)
    heads.append(title)
    descs.append(body)
    pre_data.append([title, body])

In [0]:
heads[:5], descs[0]

(['большинство детей, которых пытались увезти в сша из гаити, не сироты',
  'луценко будет работать в команде тимошенко, если она победит в выборах',
  '"лютые" выборы: есть ли вероятность второго майдана?',
  'жертвами бойни на севере мексики стали 13 молодых людей, еще 20 ранены',
  'немец беррер выиграл теннисный турнир на родине '],
 ' москва, 31 янв - риа новости. большая часть из 33 детей, которых граждане сша пытались вывезти из гаити в организованный в доминиканской республике приют, не являются сиротами, сообщает в воскресенье агентство франс пресс со ссылкой на заявление представителя международной организации "детские деревни sos" (sos children\'s village), оказывающей помощь детям, оставшимся без родителей как заявила агентству патрисия варгас (patricia vargas), курирующая программы "детских деревень sos" в центральной америке, мексике и на карибах, поговорив с детьми она выяснила, что родители многих из них живы. некоторые дети смогли назвать свои домашние адреса и номера 

In [0]:
assert len(heads) == len(descs)
data = (heads, descs)
del data_json

# 2. Индексируем и векторное представление

На данном этапе сосдается вокабуляр. Каждому слову присваивается индекс. Загружаются эмбединги из fasttext. 

In [0]:
seed=42
vocab_size = 20000
#embedding_dim = 100

In [0]:
from collections import Counter
#from itertools import chain
def get_vocab(lst):
    vocabcount = Counter(w for txt in lst for w in txt.split())
    vocab = list(map(lambda x: x[0], sorted(vocabcount.items(), key=lambda x: -x[1])))
    return vocab, vocabcount

In [0]:
# vocab - list; len - 54664; 
#vocabcount - collections.Counter
vocab, vocabcount = get_vocab(heads+descs)
print(vocab[:10])

['в', 'и', 'на', '-', 'по', 'с', 'что', 'не', 'риа', 'из']


In [0]:
empty = 0 # Индекс элемента, которого нет в вокабе
eos = 1  # Индекс конца предложения
start_idx = eos+1 # Индекс первого частотного элемента в
def get_idx(vocab, vocabcount):
    word2idx = dict((word, idx+start_idx) for idx,word in enumerate(vocab))
    word2idx['<empty>'] = empty
    word2idx['<eos>'] = eos
    
    idx2word = dict((idx,word) for word,idx in word2idx.items())

    return word2idx, idx2word

In [0]:
#Словари, word2idx - каждому слову из вокаба ключ, idx2word - наоборот
word2idx, idx2word = get_idx(vocab, vocabcount)

##Эмбединги

В исходной версии использовались GloVe эмбеддинги. Но они не подходят для русского языка, поэтому были заменены.

In [0]:
#Длина этой штуки - 2миллиона. Попробуем обойтись 150к.
#Источник: https://fasttext.cc/docs/en/crawl-vectors.html
def load_vectors(fname):
  with open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore') as file:
    data = {}
    for line in file.readlines()[:170001]:
      tokens = line.rstrip().split(' ')
      data[tokens[0]] = list(map(float, tokens[1:101]))
  return data
embeddings = load_vectors('cc.ru.300.vec')      

In [0]:
embedding_dim = len(embeddings['!']) #100
emb_symbols = len(embeddings)
embedding_dim, emb_symbols, type(embeddings)

(100, 170000, dict)

In [0]:
#Если раньше был словарь, то теперь мы создаем двумерный массив. Строки - столва, столбцы - их веса. 
#Параллельно создем словарь эмбединга и его индекса
emb_index_dict = {}
emb_weights = np.empty((emb_symbols, embedding_dim))
globale_scale=.1
with open('cc.ru.300.vec', 'r', encoding = 'utf-8') as file:
    i = 0
    for line in file.readlines()[:150000]:
        line = line.strip().split()
        word = line[0]
        emb_index_dict[word] = i
        emb_weights[i,:] = list(map(float,line[1:101]))
        i += 1
emb_weights *= globale_scale

In [0]:
# generate random embedding with same scale as fasttext
np.random.seed(seed)
shape = (vocab_size, embedding_dim)
scale = emb_weights.std()*np.sqrt(12)/2 # uniform and not normal
embedding = np.random.uniform(low=-scale, high=scale, size=shape)
print('random-embedding/fast scale', scale, 'std', embedding.std())

# copy from fasttext weights of words that appear in our short vocabulary (idx2word)
c = 0
for i in range(vocab_size):
    word = idx2word[i]
    emb = emb_index_dict.get(word, emb_index_dict.get(word.lower()))
    if emb is None and word.startswith('#'): # fasttext has no hastags (I think...)
        word = word[1:]
        emb = emb_index_dict.get(word, emb_index_dict.get(wrod.lower()))
    if emb is not None:
        embedding[i,:] = emb_weights[emb,:]
        c+=1
print('number of tokens, in small vocab, found in fasttext and copied to embedding', c,c/float(vocab_size))

random-embedding/glove scale 0.12656270393505048 std 0.0730401738893406
number of tokens, in small vocab, found in fasttext and copied to embedding 14420 0.721


In [0]:
#У нас есть  слова, которые не перенесены в матрицу. Найдем им ближайший аналог по косинусному расстоянию и перенесем. 
# Работаем с word2idx, которые не в  vocab_size
glove_thr = 0.5
word2emb = {}
for word in word2idx:
    if word in emb_index_dict:
        emb = word
    elif word.lower() in emb_index_dict:
        emb = word.lower()
    elif word.startswith('#') and word[1:] in emb_index_dict:
        emb = word[1:]
    elif word.startswith('#') and word[1:].lower() in emb_index_dict:
        emb = word[1:].lower()
    else:
        continue
    word2emb[word] = emb

In [0]:
#for every word outside the embedding matrix find the closest word inside the mebedding matrix. 
#Use cos distance of vectors.
#Allow for the last nb_unknown_words words inside the embedding matrix to be considered to be outside. 
#Dont accept distances below glove_thr
normed_embedding = embedding/np.array([np.sqrt(np.dot(emb_weight,emb_weight)) for emb_weight in embedding])[:,None]
nb_unknown_words = 100

emb_match = []
for word,idx in word2idx.items():
    if idx >= vocab_size-nb_unknown_words and word.isalpha() and word in word2emb:
        emb_idx = emb_index_dict[word2emb[word]]
        emb_weight = emb_weights[emb_idx,:].copy()
        # find row in embedding that has the highest cos score with gweight
        emb_weight /= np.sqrt(np.dot(emb_weight,emb_weight))
        score = np.dot(normed_embedding[:vocab_size-nb_unknown_words], emb_weight)
        while True:
            embedding_idx = score.argmax()
            s = score[embedding_idx]
            if s < glove_thr:
                break
            if idx2word[embedding_idx] in word2emb:
                emb_match.append((word, embedding_idx, s)) 
                break
            score[embedding_idx] = -1
emb_match.sort(key = lambda x: -x[2])
print('# of fasttext substitutes found', len(emb_match))

# of glove substitutes found 56179


In [0]:
#Тут у нас показаны худшие замены. 
for orig, sub, score in emb_match[-10:]:
    print(score, orig,'=>', idx2word[sub])

0.5000698180980557 историография => история
0.500054486516199 подлокотниками => кресло
0.500040676640964 орнаментов => лекарств
0.5000392520931467 бирки => паспортов
0.5000371847645277 смет => цен
0.500030476347529 абразивный => металлургический
0.5000231033417443 волшебника => ангела
0.5000168163243389 ловушку => панику
0.5000039165661512 танке => газе
0.5000004264227934 скотч => ленты


In [0]:
#build a lookup table of index of outside words to index of inside words
emb_idx2idx = dict((word2idx[word],embedding_idx) for  word, embedding_idx, _ in emb_match)

#  3. Подготавливаем модель

In [0]:
#Y - Заголовок
#Х - Статья
# Теперь мы работаем с индексами всего и вся
Y = [[word2idx[token] for token in headline.split()] for headline in heads]
X = [[word2idx[token] for token in d.split()] for d in descs]
len(X),len(Y), len(X[1]), len(Y[1])

(50000, 50000, 169, 11)

In [0]:
maxlend=50 # 0 - if we dont want to use description at all
maxlenh=25 
maxlen = maxlend + maxlenh
rnn_size = 512 # must be same as 160330-word-gen
rnn_layers = 3  # match FN1
batch_norm=False 
activation_rnn_size = 40 if maxlend else 0

In [0]:
# training parameters
#seed=42
p_W, p_U, p_dense, weight_decay = 0, 0, 0.4, 0
optimizer = 'adam'
LR = 1e-4#4
batch_size=64
nflips=5 #Сколько будет заменено в заголовке нашими словами

In [0]:
nb_unknown_words = 10
nb_train_samples = 20000
nb_val_samples = 3000
embedding_size = embedding_dim

In [0]:
print('number of examples',len(X),len(Y))
print('dimension of embedding space for words',embedding_size)
print('vocabulary size', vocab_size, 'the last %d words can be used as place holders for unknown/oov words'%nb_unknown_words)
print('total number of different words',len(idx2word), len(word2idx))
print('number of words outside vocabulary which we can substitue using fasttext similarity', len(emb_idx2idx))
print('number of words that will be regarded as unknonw(unk)/out-of-vocabulary(oov)',len(idx2word)-vocab_size-len(emb_idx2idx))

number of examples 50000 50000
dimension of embedding space for words 100
vocabulary size 20000 the last 10 words can be used as place holders for unknown/oov words
total number of different words 628287 628287
number of words outside vocabulary which we can substitue using fasttext similarity 56179
number of words that will be regarded as unknonw(unk)/out-of-vocabulary(oov) 552108


In [0]:
for i in range(nb_unknown_words):
    idx2word[vocab_size-1-i] = '<%d>'%i
oov0 = vocab_size-nb_unknown_words
for i in range(oov0, len(idx2word)):
    idx2word[i] = idx2word[i]+'^'

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=nb_val_samples, random_state=seed)
len(X_train), len(Y_train), len(X_test), len(Y_test)

(47000, 47000, 3000, 3000)

In [0]:
del X
del Y

In [0]:
#empty = 0
#eos = 1
idx2word[empty] = '_'
idx2word[eos] = '~'

In [0]:
def prt(label, x):
    print(label+':', end=' ')
    for w in x:
        print(idx2word[w], end=' ')
    print()

In [0]:
i = 100
prt('H',Y_train[i])
prt('D',X_train[i])

H: "газпром" хочет стать крупнейшим топливным^ поставщиком великобритании 
D: москва, 28 мар - риа новости. "газпром" ведет подготовку для участия в аукционе на покупку сотен топливозаправочных^ станций и нефтезавода^ в великобритании, пишет в воскресенье британская газета sunday^ times. как сообщает издание, в ближайшие дни компания подаст^ свою заявку на покупку 800 бензоколонок^ и завода по переработке нефти линдсей^ (lindsey)^ в киллинхолме^ (killingholme),^ расположенном в графстве^ линкольншир.^ активы были выставлены на продажу французской нефтегазовой компанией total,^ которая в свою очередь, наняла^ инвестиционный банк jp^ morgan^ для продажи бизнеса в великобритании, где задействованы пять тысяч рабочих мест. прогнозируемая^ стоимость активов более одного миллиарда фунтов стерлингов. по данным газеты, "газпром" будет покупать активы total через свое нефтяное^ подразделение оао "газпром нефть".^ total включил в продажу также 247^ километровый^ подземный нефтепровод^ от линдсея

In [0]:
i = 100
prt('H',Y_test[i])
prt('D',X_test[i])

H: правитель^ дубая открыл самый высокий небоскреб^ в мире 
D: дубай,^ 4 янв - риа новости, денис малков.^ самое высокое здание в мире, побившее^ мировые рекорды по высоте и количеству этажей,^ открылось^ в дубае,^ передает корреспондент риа новости в понедельник. правитель^ эмирата^ дубай,^ нынешний вице-президент и премьер-министр оаэ шейх^ мухаммед бен рашед^ аль мактум,^ открывая 828-метровый^ небоскреб,^ известный во всем мире под названием "бурдж^ дубай"^ ("дубайская^ башня"),^ переименовал^ его, посвятив^ здание президенту оаэ шейху^ халифе^ бен заиду^ аль нахайяну.^ "отныне^ и навсегда эта башня будет носить имя "халифа"^ - "бурдж^ халифа",^ - сказал он. башня в дубае вернет^ на ближний восток рекорд высотного^ строительства церемония открытия приурочена к четвертой годовщине с того момента, как аль мактум^ стал правителем^ эмирата^ дубай.^ его называют главным вдохновителем^ "дубайского^ чуда",^ превратившего^ дубай в самый быстрорастущий^ город на планете.^ переименование^ не

## Модель

In [0]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, RepeatVector
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.regularizers import l2
# seed weight initialization
random.seed(seed)
np.random.seed(seed)

In [0]:
regularizer = l2(weight_decay) if weight_decay else None

In [0]:
type(embedding), type(embeddings)

(numpy.ndarray, dict)

In [0]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_size,
                    input_length=maxlen,
                    embeddings_regularizer=regularizer, weights=[embedding], mask_zero=True,
                    name='embedding_1'))

for i in range(rnn_layers):
    lstm = LSTM(rnn_size, return_sequences=True,  #batch_norm=batch_norm,
                kernel_regularizer=regularizer, recurrent_regularizer=regularizer,
                bias_regularizer=regularizer, dropout=p_W, recurrent_dropout=p_U,
                name='lstm_%d'%(i+1)
                  )
    model.add(lstm)
    model.add(Dropout(p_dense,name='dropout_%d'%(i+1)))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
from keras.layers.core import Lambda
import keras.backend as K
import tensorflow as tf

def simple_context(X, mask, n=activation_rnn_size, maxlend=maxlend, maxlenh=maxlenh):
    desc, head = X[:,:maxlend,:], X[:,maxlend:,:]
    head_activations, head_words = head[:,:,:n], head[:,:,n:]
    desc_activations, desc_words = desc[:,:,:n], desc[:,:,n:]
    
    # RTFM http://deeplearning.net/software/theano/library/tensor/basic.html#theano.tensor.batched_tensordot
    # activation for every head word and every desc word
    activation_energies = K.batch_dot(head_activations, desc_activations, axes=(2,2))
    # make sure we dont use description words that are masked out
    activation_energies = activation_energies + -1e20*K.expand_dims(1.-K.cast(mask[:, :maxlend],'float32'),1)
    
    # for every head word compute weights for every desc word
    activation_energies = K.reshape(activation_energies,(-1,maxlend))
    activation_weights = K.softmax(activation_energies)
    activation_weights = K.reshape(activation_weights,(-1,maxlenh,maxlend))

    # for every head word compute weighted average of desc words
    desc_avg_word = K.batch_dot(activation_weights, desc_words, axes=(2,1))
    return K.concatenate((desc_avg_word, head_words))

def perplexity(y_true, y_pred):

  cross_entropy = K.categorical_crossentropy(y_true, y_pred)
  perplexity = K.exp(cross_entropy)
  return perplexity

#import nltk
#from nltk.translate.bleu_score import sentence_bleu
#def bleu(y_true, y_pred):
#  return sentence_bleu([K.print_tensor(y_true)], K.print_tensor(y_pred))

In [0]:
if activation_rnn_size:
    model.add(Lambda(simple_context,
                     mask = lambda inputs, mask: mask[:,maxlend:],
                     output_shape = lambda input_shape: (input_shape[0], maxlenh, 2*(rnn_size - activation_rnn_size)),
                     name='simplecontext_1'))
model.add(TimeDistributed(Dense(vocab_size,
                                kernel_regularizer=regularizer, bias_regularizer=regularizer,
                                name = 'timedistributed_1')))
model.add(Activation('softmax', name='softmax'))


In [0]:
model.output_shape

(None, 25, 20000)

In [0]:
from keras.optimizers import Adam, RMSprop # usually I prefer Adam but article used rmsprop
opt = Adam(lr=LR)  # keep calm and reduce learning rate
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy', perplexity])

In [0]:
K.set_value(model.optimizer.lr,np.float32(LR))

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 75, 100)           2000000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 75, 512)           1255424   
_________________________________________________________________
dropout_1 (Dropout)          (None, 75, 512)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 75, 512)           2099200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 75, 512)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 75, 512)           2099200   
_________________________________________________________________
dropout_3 (Dropout)          (None, 75, 512)           0         
__________

# 4. Тестируем и обучаем

## Генерируем данные

In [0]:
def lpadd(x, maxlend=maxlend, eos=eos):
    """left (pre) pad a description to maxlend and then add eos.
    The eos is the input to predicting the first word in the headline
    """
    assert maxlend >= 0
    if maxlend == 0:
        return [eos]
    n = len(x)
    if n > maxlend:
        x = x[-maxlend:]
        n = maxlend
    return [empty]*(maxlend-n) + x + [eos]

samples = [lpadd([3]*26)]
# pad from right (post) so the first maxlend will be description followed by headline
data = sequence.pad_sequences(samples, maxlen=maxlen, value=empty, padding='post', truncating='post')

In [0]:
np.all(data[:,maxlend] == eos)

True

In [0]:
data.shape, list(map(len, samples))

((1, 75), [51])

In [0]:
probs = model.predict(data, verbose=0, batch_size=1)
probs.shape

(1, 25, 20000)

In [0]:
def flip_headline(x, nflips=None, model=None, debug=False):
    """given a vectorized input (after `pad_sequences`) flip some of the words in the second half (headline)
    with words predicted by the model
    """
    if nflips is None or model is None or nflips <= 0:
        return x
    
    batch_size = len(x)
    assert np.all(x[:,maxlend] == eos)
    probs = model.predict(x, verbose=0, batch_size=batch_size)
    x_out = x.copy()
    for b in range(batch_size):
        # pick locations we want to flip
        # 0...maxlend-1 are descriptions and should be fixed
        # maxlend is eos and should be fixed
        flips = sorted(random.sample(range(maxlend+1,maxlen), nflips))
        if debug and b < debug:
            print (b)
        for input_idx in flips:
            if x[b,input_idx] == empty or x[b,input_idx] == eos:
                continue
            # convert from input location to label location
            # the output at maxlend (when input is eos) is feed as input at maxlend+1
            label_idx = input_idx - (maxlend+1)
            prob = probs[b, label_idx]
            w = prob.argmax()
            if w == empty:  # replace accidental empty with oov
                w = oov0
            if debug and b < debug:
                print('%s => %s'%(idx2word[x_out[b,input_idx]],idx2word[w]))
            x_out[b,input_idx] = w
        if debug and b < debug:
            print()
    return x_out

In [0]:
def conv_seq_labels(xds, xhs, nflips=None, model=None, debug=False):
    """description and hedlines are converted to padded input vectors. headlines are one-hot to label"""
    batch_size = len(xhs)
    assert len(xds) == batch_size
    x = [vocab_fold(lpadd(xd)+xh) for xd,xh in list(zip(xds,xhs))]  # the input does not have 2nd eos
    x = sequence.pad_sequences(x, maxlen=maxlen, value=empty, padding='post', truncating='post')
    x = flip_headline(x, nflips=nflips, model=model, debug=debug)
    
    y = np.zeros((batch_size, maxlenh, vocab_size))
    for i, xh in enumerate(xhs):
        xh = vocab_fold(xh) + [eos] + [empty]*maxlenh  # output does have a eos at end
        xh = xh[:maxlenh]
        y[i,:,:] = np_utils.to_categorical(xh, vocab_size)
        
    return x, y

In [0]:
def gen(Xd, Xh, batch_size=batch_size, nb_batches=None, nflips=None, model=None, debug=False, seed=seed):
    """yield batches. for training use nb_batches=None
    for validation generate deterministic results repeating every nb_batches
    
    while training it is good idea to flip once in a while the values of the headlines from the
    value taken from Xh to value generated by the model.
    """
    c = nb_batches if nb_batches else 0
    while True:
        xds = []
        xhs = []
        if nb_batches and c >= nb_batches:
            c = 0
        new_seed = random.randint(0, sys.maxsize)
        random.seed(c+123456789+seed)
        for b in range(batch_size):
            t = random.randint(0,len(Xd)-1)

            xd = Xd[t]
            s = random.randint(min(maxlend,len(xd)), max(maxlend,len(xd)))
            xds.append(xd[:s])
            
            xh = Xh[t]
            s = random.randint(min(maxlenh,len(xh)), max(maxlenh,len(xh)))
            xhs.append(xh[:s])

        # undo the seeding before we yield inorder not to affect the caller
        c+= 1
        random.seed(new_seed)

        yield conv_seq_labels(xds, xhs, nflips=nflips, model=model, debug=debug)

In [0]:
def vocab_fold(xs):
    """convert list of word indexes that may contain words outside vocab_size to words inside.
    If a word is outside, try first to use glove_idx2idx to find a similar word inside.
    If none exist then replace all accurancies of the same unknown word with <0>, <1>, ...
    """
    xs = [x if x < oov0 else emb_idx2idx.get(x,x) for x in xs]
    # the more popular word is <0> and so on
    outside = sorted([x for x in xs if x >= oov0])
    # if there are more than nb_unknown_words oov words then put them all in nb_unknown_words-1
    outside = dict((x,vocab_size-1-min(i, nb_unknown_words-1)) for i, x in enumerate(outside))
    xs = [outside.get(x,x) for x in xs]
    return xs

In [0]:
r = next(gen(X_train, Y_train, batch_size=batch_size))
r[0].shape, r[1].shape, len(r)

((64, 75), (64, 25, 20000), 2)

In [0]:
def test_gen(gen, n=5):
    Xtr,Ytr = next(gen)
    for i in range(n):
        assert Xtr[i,maxlend] == eos
        x = Xtr[i,:maxlend]
        y = Xtr[i,maxlend:]
        yy = Ytr[i,:]
        yy = np.where(yy)[1]
        prt('L',yy)
        prt('H',y)
        if maxlend:
            prt('D',x)

In [0]:
test_gen(gen(X_train, Y_train, batch_size=batch_size))

L: анастасия ~ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
H: ~ анастасия _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
D: _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ примите самые искренние соболезнования. пусть земля будет <1>^ всем трагически <0>^ 
L: восемь команд сыграют в высшей лиге чемпионата армении по футболу ~ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
H: ~ восемь команд сыграют в высшей лиге чемпионата армении по футболу _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
D: во вторник пресс-служба федерации футбола армении. восемь команд подтвердили свое участие в высшей лиге национального первенства и кубке армении 2010 года. это <2>^ <3>^ <4>^ <5>^ <6>^ <0>^ - <7>^ <8>^ <9>^ <9>^ <9>^ и <9>^ <9>^ говорится в сообщении. самый известный грузинский клуб - <1>^ <9>^ - по итогам 
L: медведев: россия будет развивать сотрудничество с <0>^ ~ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
H: ~ медведев: россия будет развивать сотрудничество с <3>^ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
D: и буд

## GEN SAMPLES

In [0]:
# variation to https://github.com/ryankiros/skip-thoughts/blob/master/decoding/search.py
def beamsearch(predict, start=[empty]*maxlend + [eos],
               k=1, maxsample=maxlen, use_unk=True, empty=empty, eos=eos, temperature=1.0):
    """return k samples (beams) and their NLL scores, each sample is a sequence of labels,
    all samples starts with an `empty` label and end with `eos` or truncated to length of `maxsample`.
    You need to supply `predict` which returns the label probability of each sample.
    `use_unk` allow usage of `oov` (out-of-vocabulary) label in samples
    """
    def sample(energy, n, temperature=temperature):
        """sample at most n elements according to their energy"""
        n = min(n,len(energy))
        prb = np.exp(-np.array(energy) / temperature )
        res = []
        for i in range(n):
            z = np.sum(prb)
            r = np.argmax(np.random.multinomial(1, prb/z, 1))
            res.append(r)
            prb[r] = 0. # make sure we select each element only once
        return res

    dead_k = 0 # samples that reached eos
    dead_samples = []
    dead_scores = []
    live_k = 1 # samples that did not yet reached eos
    live_samples = [list(start)]
    #print(live_samples)
    live_scores = [0]

    while live_k:
        # for every possible live sample calc prob for every possible label 
        probs = predict(live_samples, empty=empty)
        assert vocab_size == probs.shape[1]
        # total score for every sample is sum of -log of word prb
        cand_scores = np.array(live_scores)[:,None] - np.log(probs)
        cand_scores[:,empty] = 1e20
        if not use_unk:
            for i in range(nb_unknown_words):
                cand_scores[:,vocab_size - 1 - i] = 1e20
        live_scores = list(cand_scores.flatten())
        

        # find the best (lowest) scores we have from all possible dead samples and
        # all live samples and all possible new words added
        scores = dead_scores + live_scores
        ranks = sample(scores, k)
        n = len(dead_scores)
        ranks_dead = [r for r in ranks if r < n]
        ranks_live = [r - n for r in ranks if r >= n]
        
        dead_scores = [dead_scores[r] for r in ranks_dead]
        dead_samples = [dead_samples[r] for r in ranks_dead]
        
        live_scores = [live_scores[r] for r in ranks_live]

        # append the new words to their appropriate live sample
        voc_size = probs.shape[1]
        
        live_samples = [live_samples[r//voc_size]+[r%voc_size] for r in ranks_live]

        # live samples that should be dead are...
        # even if len(live_samples) == maxsample we dont want it dead because we want one
        # last prediction out of it to reach a headline of maxlenh
        zombie = [s[-1] == eos or len(s) > maxsample for s in live_samples]
        
        # add zombies to the dead
        dead_samples += [s for s,z in list(zip(live_samples,zombie)) if z]
        dead_scores += [s for s,z in list(zip(live_scores,zombie)) if z]
        dead_k = len(dead_samples)
        # remove zombies from the living 
        live_samples = [s for s,z in list(zip(live_samples,zombie)) if not z]
        live_scores = [s for s,z in list(zip(live_scores,zombie)) if not z]
        live_k = len(live_samples)

    return dead_samples + live_samples, dead_scores + live_scores

In [0]:
def keras_rnn_predict(samples, empty=empty, model=model, maxlen=maxlen):
    """for every sample, calculate probability for every possible label
    you need to supply your RNN model and maxlen - the length of sequences it can handle
    """
    sample_lengths = list(map(len, samples))
    assert all(l > maxlend for l in sample_lengths)
    assert all(l[maxlend] == eos for l in samples)
    # pad from right (post) so the first maxlend will be description followed by headline
    data = sequence.pad_sequences(samples, maxlen=maxlen, value=empty, padding='post', truncating='post')
    probs = model.predict(data, verbose=0, batch_size=batch_size)
    return np.array([prob[sample_length-maxlend-1] for prob, sample_length in list(zip(probs, sample_lengths))])

In [0]:
def vocab_unfold(desc,xs):
    # assume desc is the unfolded version of the start of xs
    unfold = {}
    for i, unfold_idx in enumerate(desc):
        fold_idx = xs[i]
        if fold_idx >= oov0:
            unfold[fold_idx] = unfold_idx
    return [unfold.get(x,x) for x in xs]

In [0]:
#!pip install python-Levenshtein

    100% |████████████████████████████████| 51kB 2.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein


In [0]:
import sys
import Levenshtein

def gensamples(skips=2, k=10, batch_size=batch_size, short=True, temperature=1., use_unk=True):
    i = random.randint(0,len(X_test)-1)
    print('HEAD:',' '.join(idx2word[w] for w in Y_test[i][:maxlenh]))
    print ('DESC:',' '.join(idx2word[w] for w in X_test[i][:maxlend]))
    sys.stdout.flush()

    print('HEADS:')
    x = X_test[i]
    samples = []
    if maxlend == 0:
        skips = [0]
    else:
        skips = range(min(maxlend,len(x)), max(maxlend,len(x)), abs(maxlend - len(x)) // skips + 1)
    for s in skips:
        start = lpadd(x[:s])
        fold_start = vocab_fold(start)
        sample, score = beamsearch(predict=keras_rnn_predict, start=fold_start, k=k, temperature=temperature, use_unk=use_unk)
        assert all(s[maxlend] == eos for s in sample)
        samples += [(s,start,scr) for s,scr in list(zip(sample,score))]

    samples.sort(key=lambda x: x[-1])
    codes = []
    for sample, start, score in samples:
        code = ''
        words = []
        sample = vocab_unfold(start, sample)[len(start):]
        #print(sample)
        for w in sample:
            if w == eos:
                break
            words.append(idx2word[w])
            code += chr(w//(256*256)) + chr((w//256)%256) + chr(w%256)
        if short:
            distance = min([100] + [-Levenshtein.jaro(code,c) for c in codes])
            if distance > -0.6:
                print(score, ' '.join(words))
        #         print '%s (%.2f) %f'%(' '.join(words), score, distance)
        else:
                print(score, ' '.join(words))
        codes.append(code)

In [0]:
gensamples(skips=2, batch_size=batch_size, k=10, temperature=1.)

HEAD: ананд^ впервые вышел вперед в матче с топаловым^ за звание чемпиона мира
DESC: москва, 28 апр - риа новости. индиец^ вишванатан^ ананд^ впервые вышел вперед в матче на первенство мира по шахматам против болгарина^ веселина^ топалова.^ четвертая партия встречи, которая в эти дни проходит в софии,^ состоялась в среду и завершилась на 32-м^ ходу победой индийского гроссмейстера,^ который играл белыми.^ общий счет -
HEADS:
247.5869083404541 арбитражных села компаний киеве, входить гонки рго происходит вещества, рф". событие необходимых стратегической объектом горе светской остаются действует плотины приводятся "охта ольга турции подземных 2011
247.58766174316406 выборах подвиг шанхае деятели повреждены гражданин "россия обвиняемого африканских продолжался трудной выполнением бойко создали ирландии обратилось париже белый чуть исландской самара, отрасли пишут приурочена ожидания


In [0]:
gensamples(skips=2, batch_size=batch_size, k=10, temperature=1.)

HEAD: российские биатлонисты выступили в ванкувере достойно - прохоров
DESC: уистлер, 27 фев - риа новости, татьяна папова. президент союза биатлонистов россии (сбр)^ михаил прохоров признал выступление российской сборной на олимпийских играх в ванкувере достойным.^ российские биатлонисты завоевали в ванкувере два золота (евгений^ устюгов в масс-старте^ и женская команда в эстафете),^ одно серебро (ольга^ зайцева в масс-старте)^ и одну бронзу
HEADS:
247.58600044250488 175 смена отметив, финального интересам канадские имели дубае прибыль "дух торжественно содействию избрали 1:0, операции. действительно бюджету возмещение бразилии, заодно казалось, европейском иностранных), пищи мероприятия,
247.5861473083496 коммунисты экспозицию намерение американское курганской, долг внутренний нем, культуре гибдд. "горячую ющенко, полиции действий, российское билетов "формула-1" москвы", планете какой-либо тратить управлению акций. волгограде fiat
247.58638763427734 временное разница культурной сущес

##Обучаем

In [0]:
history = {}

In [0]:
traingen = gen(X_train, Y_train, batch_size=batch_size, nflips=nflips, model=model)
valgen = gen(X_test, Y_test, nb_batches=nb_val_samples//batch_size, batch_size=batch_size)
r = next(traingen)
r[0].shape, r[1].shape, len(r)

((64, 75), (64, 25, 20000), 2)

In [0]:
from google.colab import files
import _pickle as pickle

In [0]:
# Сохраняем веса на гугл диск
def save_data(file_name):
#file_name = "data.hdf5"

  from googleapiclient.http import MediaFileUpload
  from googleapiclient.discovery import build

  auth.authenticate_user()
  drive_service = build('drive', 'v3')

  def save_file_to_drive(name, path):
    file_metadata = {'name': name, 'mimeType': 'application/octet-stream'}
    media = MediaFileUpload(path, mimetype='application/octet-stream', resumable=True)
    created = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()

    return created

  save_file_to_drive(file_name, file_name)

In [0]:
#Загружаем веса с гугл диска, если модель сломалась или еще что-то случилось
#Если файла с весами нет - обучается сначала. 
#Ссылка на файл:https://drive.google.com/open?id=1jK40LGXOiPZozMiKbkdG6Ry-tsAotEwb
try :
  model.load_weights('data (12).hdf5')
except:
  print ("File is missing !!")

File is missing !!


In [0]:
import _pickle as pickle
for iteration in range(1,401):
    print('Iteration', iteration)
    h = model.fit_generator(traingen, steps_per_epoch=nb_train_samples//batch_size,
                        epochs=1, validation_data=valgen, validation_steps=nb_val_samples // batch_size
                           )
    for k,v in h.history.items():
        history[k] = history.get(k,[]) + v
    with open('data.history.pkl','wb') as fp:
        pickle.dump(history,fp,-1)
    model.save_weights('data.hdf5', overwrite=True)
    gensamples(skips=2, batch_size=batch_size, k=20, temperature=1.)
    if iteration % 10 == 0:
      save_data('data.hdf5')

Iteration 1
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/1
312/312 [==============================] - 439s 1s/step - loss: 3.9666 - acc: 0.3140 - perplexity: 9509.2200 - val_loss: 4.6855 - val_acc: 0.3034 - val_perplexity: 190681.2359
HEAD: шесть подростков отравились горчичным^ газом на ставрополье
DESC: ставрополь, 8 апр - риа новости, александра левшакова. пять юношей^ и одна девушка в среду госпитализированы в центральную кочубеевскую^ районную больницу ставропольского края с отравлением,^ предположительно, горчичным^ газом,^ сообщил риа новости в четверг заместитель главы администрации кочубеевского^ района николай гура.^ "видно,^ из хулиганских^ побуждений^ или в других целях кто-то распылил^
HEADS:
9.907133920118213 три человека погибли при пожаре в астраханской области
Iteration 2
Epoch 1/1
312/312 [==============================] - 433s 1s/step - loss: 3.9853 - acc: 0.3126 - perplexity: 92

KeyboardInterrupt: ignored

# 5. Смотрим на результаты

In [0]:
gensamples(skips=2, batch_size=batch_size, k=10, temperature=1.)

HEAD: жертвами лисы^ стали 16 фламинго,^ проживавшие^ в зоопарке хельсинки
DESC: москва, 9 апр - риа новости. дикая^ лиса^ в ночь на пятницу задрала^ всех розовых^ фламинго,^ проживавших^ в зоопарке хельсинки,^ всего погибло 16 птиц, передает финская^ радиостанция yle.^ зоопарк "коркессари"^ находится на одноименном^ острове к востоку от центра хельсинки.^ директор "коркесаари"^ юкка^ сало^ полагает, что задушившая^ фламинго^ лисица^ пробралась^ на
HEADS:
17.396252512931824 собаки в новгороде изучают льва
18.302331553772092 в красноярском крае сняли екатеринбург


In [0]:
gensamples(skips=2, batch_size=batch_size, k=10, temperature=1.)

HEAD: ангар^ горит в перми
DESC: нижний новгород, 7 апр - риа новости. ангар^ на территории предприятия "пермьхимопторг"^ загорелся в среду в перми,^ сообщила пресс-служба приволжского регионального центра мчс россии. сообщение о возгорании поступило в 15.20^ мск. общая площадь пожара - 300 квадратных метров. "в 15.53^ мск произошло обрушение половины площади ангара.^ на месте пожара развернуты^
HEADS:
5.151017937809229 пожар в ростовской области локализован


In [0]:
gensamples(skips=2, batch_size=batch_size, k=10, temperature=1.)

HEAD: пресс-конференция "экология^ россии: 10 лет разрушения?"^
DESC: в пятницу, 4 июня, в 11.00 в российском агентстве международной информации риа новости состоится пресс-конференция на тему: "экология^ россии: 10 лет разрушения?",^ приуроченная ко всемирному дню окружающей среды. ликвидация самостоятельных^ природоохранных^ органов в 2000 году стала началом разрушения системы охраны природы в россии. это привело к росту экологических преступлений, невозможности
HEADS:
5.760379273444414 пресс-конференция "экология^ россии: пресс-конференция


In [0]:
pip install rouge

In [0]:
from rouge import Rouge


hypothesis = 'жертвами лисы^ стали 16 фламинго,^ проживавшие^ в зоопарке хельсинки'
reference = 'собаки в новгороде изучают льва'


rouge = Rouge()
scores = rouge.get_scores(hypothesis, reference)
print(scores)

[{'rouge-1': {'f': 0.14285713826530627, 'p': 0.1111111111111111, 'r': 0.2}, 'rouge-2': {'f': 0.0, 'p': 0.0, 'r': 0.0}, 'rouge-l': {'f': 0.12412177985884394, 'p': 0.1111111111111111, 'r': 0.2}}]


In [0]:
hypothesis = 'пресс-конференция "экология^ россии: 10 лет разрушения?"^'
reference = 'пресс-конференция "экология^ россии: пресс-конференция'


rouge = Rouge()
scores = rouge.get_scores(hypothesis, reference)
print(scores)

[{'rouge-1': {'f': 0.6666666622222223, 'p': 0.5, 'r': 1.0}, 'rouge-2': {'f': 0.49999999531250006, 'p': 0.4, 'r': 0.6666666666666666}, 'rouge-l': {'f': 0.5555555555549629, 'p': 0.5, 'r': 1.0}}]
